In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from langchain import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, pipeline
 

model_id = "codellama/CodeLlama-34b-hf"

peft_model_id = "/home/ksaff/Desktop/ttyd/fine_tuning/2nd_try"
config = PeftConfig.from_pretrained(peft_model_id)

bnb_config = BitsAndBytesConfig(
                                load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16
                                )

model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map={'': 0}
)
tokenizer = AutoTokenizer.from_pretrained(
                                          config.base_model_name_or_path)

model = model.bfloat16()

/home/ksaff/miniconda3/envs/llm_jupyter/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model.safetensors.index.json
['/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model-00001-of-00007.safetensors', '/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model-00002-of-00007.safetensors', '/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model-00003-of-00007.safetensors', '/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model-00004-of-00007.safetensors', '/home/ksaff/.cache/huggingface/hub/models--codellama--CodeLlama-34b-hf/snapshots/82128714b6174570a64b3dd1f3e9c146bda26cf9/model-00005-of-00007.safetensors', '/home/ksaff/.cache/huggingface/hub/models--codellama--Cod

Loading checkpoint shards:  86%|████████▌ | 6/7 [00:17<00:02,  2.83s/it]

In [ ]:
generation_config = GenerationConfig.from_pretrained(model_id)
generation_config.max_new_tokens = 1024
# generation_config.temperature = 0.0001
# generation_config.top_p = 0.95
# generation_config.do_sample = True
# generation_config.repetition_penalty = 1.15
generation_config.max_time = 3
generation_config.pad_token_id = 2
 
text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    generation_config=generation_config,
)
 
llm = HuggingFacePipeline(pipeline=text_pipeline)

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory()
conversation_buf = ConversationChain(
    llm=llm,
    memory=memory,)

In [ ]:
database_scheme = """
CREATE TABLE Salaries
  Id INTEGER PRIMARY KEY, -- Unique ID for each employee
  EmployeeName VARCHAR, -- Name of the employee
  JobTitle VARCHAR, -- Name of employees proffesion
  BasePay NUMERIC, -- Base pay of employee
  OvertimePay NUMERIC, -- Overtime pay of employee
  OtherPay NUMERIC, -- Other pays of employee
  Benefits NUMERIC, -- Benefits of employee
  TotalPay NUMERIC, -- Total pay of employee
  TotalPayBenefits NUMERIC, -- Sum of pay benefits of employee
  Year INTEGER, -- Year data from row reffers to
"""

question = "Give me sum of all benefits in year 2013"

text = (
    f"""### Task
    Generate a SQL query to answer the following question:
    {question}
### Database Schema
    This query will run on a database whose schema is represented in this string:""" 
    +
    database_scheme
    +
f"""### SQL
    Given the database schema, here is the SQL query that answers `{question}`:
    ```sql
    """
)

In [ ]:
import sqlparse

response = conversation_buf(text)
print(f'question - {question}')
print(response['response'].split("```")[-2].replace('sql', ''))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


question = Give me sum of all benefits in year 2013

    SELECT SUM(Benefits) FROM Salaries WHERE Year = 2013
    


In [ ]:
question_2 = 'Now do the same for total pay'
response = conversation_buf(question_2)

print(f'question - {question_2}')
print(response['response'].split("```")[-2].replace('sql', ''))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


question = Now do the same for total pay

    SELECT SUM(TotalPay) FROM Salaries WHERE Year = 2013
    
